In [143]:
import pandas as pd
import numpy as np
report_sens = pd.read_csv('./result/report_sens.csv')
report_query = pd.read_csv('./result/report_query.csv')
report_DP = pd.read_csv('./result/report_DP.csv')
report_DP_limits = pd.read_csv('./result/report_DP_limits.csv')
queries = dict()
for qtype in ['triangle', '4path', '4cycle', '4star']:
    queries[qtype] = report_sens[report_sens['query'].str.startswith("q_%s"%qtype)]['query'].unique()
qtype_tex = {
    'triangle' : '$\qtri$',
    '4path' : '$\qpath$',
    '4cycle' : '$\qcycle$',
    '4star' : '$\qstar$'
}
qtypes = ['triangle', '4path', '4cycle', '4star']

In [145]:
def gen_table_meta_latex(queries, report_sens, report_query):
    qtypes = ['triangle', '4path', '4cycle', '4star']
    SETTINGS = 4
    
    #qtypes = ['triangle']
    #SETTINGS = 1
    
    tex_template = open('./result/table_meta.tex.template', 'r').read()
    filler = ''
    for qtype in qtypes:
        olevel_rows = sum([ \
                          sum([sum(len(tbl['attributes']) \
                        for tbl in eval(report_query.query('query=="%s"'%queries[qtype][i]).iloc[0]['table_dict']).values())]) \
                          for i in range(SETTINGS) ])
        filler += '\multirow{%d}{*}{%s} & '%(olevel_rows, qtype_tex[qtype])
        
        first_olevel = True
        for i in range(SETTINGS):
            if not first_olevel:
                filler += '& '
            qname = queries[qtype][i]
            info = report_query.query('query=="%s"'%qname).iloc[0]
            table_dict = eval(info['table_dict'])
            setting = '$s_%d$'%(i+1)
            qlevel_rows = sum([sum(len(tbl['attributes']) for tbl in table_dict.values())])
            median_mf = np.median([attr['mf'] for tbl in table_dict.values() for attr in tbl['attributes'].values()])
            filler += '\multirow{%d}{*}{%s} & '%(qlevel_rows, setting)
            filler += '\multirow{%d}{*}{%d} & '%(qlevel_rows, info['total_input'])
            filler += '\multirow{%d}{*}{%d} & '%(qlevel_rows, info['total_repr'])
            #filler += '\multirow{%d}{*}{%d} & '%(qlevel_rows, median_mf)
            filler += '\multirow{%d}{*}{%d} & '%(qlevel_rows, info['total_output'])
            first_qlevel = True
            for reln, reln_info in table_dict.items():
                tlevel_rows = len(reln_info['attributes'])
                if not first_qlevel:
                    filler += '& '*5
                filler += '\multirow{%d}{*}{%s} & '%(tlevel_rows, reln.replace('_', '\\_'))
                #filler += '\multirow{%d}{*}{reln} & '%(tlevel_rows)
                filler += '\multirow{%d}{*}{%d} & '%(tlevel_rows, reln_info['cnt'])
                filler += '\multirow{%d}{*}{%d} & '%(tlevel_rows, reln_info['repr'])
                first_tlevel = True
                for attr, attr_info in reln_info['attributes'].items():
                    if not first_tlevel:
                        filler += '& '*8
                    filler += '%s & '%attr
                    filler += '%d & '%attr_info['active']
                    filler += '%d & '%attr_info['repr']
                    filler += '%d '%attr_info['mf']
                    filler += '\\\\ \cline{9-12} \n'
                    first_tlevel = False
                filler += ' \cline{6-12} \n'
                first_qlevel = False
            filler += '\cline{2-12} \n'
            first_olevel = False
        filler += '\hline'
    table_tex = tex_template.replace('<filler>', filler)
    return table_tex

In [146]:
print(gen_table_meta_latex(queries, report_sens, report_query))

%\begin{table*}[t!]
%\usepackage{longtable}
%for \Yup
%\usepackage{stmaryrd}
%for \square \sqcap
%\usepackage{mathabx}
\def\qtri{q_\triangle}
\def\qpath{q_\sqcap}
\def\qcycle{q_\square}
\def\qstar{q_\Yup}
\def\tsens{TSens}
\def\elastic{Elastic}
%\begin{tabular}{|l|l|l|l|l|l|l|l|l|l|l|l|}
\begin{longtable}{|l|l|l|l|l|l|l|l|l|l|l|l|}
\hline
qtype & setting & $|IN|$ & $|\reprdomain|$ & $|OUT|$ & relations & $|IN|$ & $\reprdomain$ & attr & $\actdomain$ & $\reprdomain$ & mf \\ \hline
\endhead
\multirow{6}{*}{$\qtri$} & \multirow{6}{*}{$s_1$} & \multirow{6}{*}{13520} & \multirow{6}{*}{62257} & \multirow{6}{*}{30699} & \multirow{2}{*}{source\_348\_circle\_dabfae} & \multirow{2}{*}{6830} & \multirow{2}{*}{13857} & A & 200 & 93 & 99 \\ \cline{9-12} 
& & & & & & & & B & 200 & 149 & 99 \\ \cline{9-12} 
 \cline{6-12} 
& & & & & \multirow{2}{*}{source\_348\_circle\_4d72cf} & \multirow{2}{*}{4818} & \multirow{2}{*}{18600} & C & 149 & 93 & 76 \\ \cline{9-12} 
& & & & & & & & B & 149 & 200 & 76 \\ \cl

In [118]:
def gen_sens_table_latex(queries, selected_query, report_sens, report_query):
    tex_template = open('./result/table_sens.tex.template', 'r').read()
    filler = ''
    
    for qtype in qtypes:
        filler += '%s '%qtype_tex[qtype]
        qname = queries[qtype][selected_query[qtype]]
        for algo in ['TSens', 'Elastic']:
            val = report_sens.query('algo=="{algo}" & query=="{qname}"'.format(algo=algo, qname=qname)).iloc[0]
            filler += '& %d'%val['sensitivity']
        for algo in ['TSens', 'Elastic']:
            val = report_sens.query('algo=="{algo}" & query=="{qname}"'.format(algo=algo, qname=qname)).iloc[0]
            filler += '& %.3f'%val['time']
        filler += '& %.3f'%report_query.query('query=="{qname}"'.format(qname=qname)).iloc[0]['avg_time']
        filler += '\\\\ \hline\n'
        
    table_tex = tex_template.replace('<filler>', filler)
    return table_tex

selected_query = {
    'triangle': 2,
    '4path': 2,
    '4cycle': 2,
    '4star': 2
}

In [119]:
print(gen_sens_table_latex(queries, selected_query, report_sens, report_query))

\begin{table}[t!]
%for \Yup
%\usepackage{stmaryrd}
%for \square \sqcap
%\usepackage{mathabx}
\def\qtri{q_\triangle}
\def\qpath{q_\sqcap}
\def\qcycle{q_\square}
\def\qstar{q_\Yup}
\def\tsens{TSens}
\def\elastic{Elastic}
\begin{tabular}{|l|l|l|l|l|l|l|l|l|}
\hline
& \multicolumn{2}{c|}{LS} & \multicolumn{3}{c|}{Time (seconds)} \\ \cline{2-6}
& \tsens & \elastic & \tsens & \elastic & Evaluation \\ \hline
$\qtri$ & 76& 7524& 0.440& 0.007& 0.388\\ \hline
$\qpath$ & 115650& 225720& 0.227& 0.010& 0.178\\ \hline
$\qcycle$ & 218& 65610& 0.115& 0.005& 0.128\\ \hline
$\qstar$ & 29& 14008500& 0.371& 0.059& 0.195\\ \hline
\end{tabular}
\caption{Local sensitivity and time usage of 4 query types for TSens and Elastic. $\qtri$ is a triangle query, $\qpath$ is a path query of 4 nodes, $\qcycle$ is a 4-cycle query, $\qstar$ is a star query of 4 nodes. It also reports the query evaluation time for counting the output size.}
\end{table}


In [141]:
def gen_DP_table_latex(queries, selected_query, report_DP, report_query):
    df = report_DP.copy()
    df['time'] = df['avg_time']
    df.loc[df['nosy_ans'] < 0, 'nosy_ans'] = 0 
    df['relerror'] = df.apply(lambda x: np.median(np.absolute(np.array(eval(x['nosy_ans_list'])) - x['true_ans']) / x['true_ans']), axis=1)
    df['relbias'] = df.apply(lambda x: np.median(np.absolute(np.array(eval(x['bias_ans_list'])) - x['true_ans']) / x['true_ans']), axis=1)
    
    tex_template = open('./result/table_dp.tex.template', 'r').read()
    filler = ''
    
    for qtype in qtypes:
        filler += '%s &'%qtype_tex[qtype]
        qname = queries[qtype][selected_query[qtype]]
        filler += '%d &'%report_query.query('query=="{qname}"'.format(qname=qname)).iloc[0]['total_output']
        first_alevel = True
        for algo in ['TSensDP', 'PrivateSQL']:
            if not first_alevel:
                filler += '&'*2
            filler += ' %s '%algo
            val = df.query('algo=="{algo}" & query=="{qname}"'.format(algo=algo, qname=qname)).iloc[0]
            filler += '& {err:.2f}\\%  & {bias:.2f}\\% & {GS:.0f} & {time:.3f}'.format(GS=val['gsens'], bias=val['relbias']*100, err=val['relerror']*100, time=val['avg_time'])
            filler += '\\\\ \cline{3-7}'
            first_alevel = False
        filler += '\hline'        
    table_tex = tex_template.replace('<filler>', filler)
    return table_tex

selected_query = {
    'triangle': 2,
    '4path': 2,
    '4cycle': 2,
    '4star': 2
}

In [142]:
print(gen_DP_table_latex(queries, selected_query, report_DP, report_query))

\begin{table}[t!]
%for \Yup
%\usepackage{stmaryrd}
%for \square \sqcap
%\usepackage{mathabx}
\def\qtri{q_\triangle}
\def\qpath{q_\sqcap}
\def\qcycle{q_\square}
\def\qstar{q_\Yup}
\def\tsensdp{TSensDP}
\def\privatesql{PrivateSQL}
\begin{tabular}{|l|l|l|l|l|l|l|l|l|}
\hline
 Query & $|Q(D)|$ & Algorithm & \multicolumn{1}{c|}{Error} & \multicolumn{1}{c|}{Bias} & \multicolumn{1}{c|}{GS} & \multicolumn{1}{c|}{Time}\\ \hline
$\qtri$ &20751 & TSensDP & 0.99\%  & 0.72\% & 54 & 0.515\\ \cline{3-7}&& PrivateSQL & 43.46\%  & 0.00\% & 4950 & 0.223\\ \cline{3-7}\hline$\qpath$ &2012183 & TSensDP & 4.16\%  & 0.71\% & 28885 & 1.007\\ \cline{3-7}&& PrivateSQL & 5.89\%  & 0.00\% & 71280 & 1.790\\ \cline{3-7}\hline$\qcycle$ &3825 & TSensDP & 63.67\%  & 8.37\% & 548 & 0.312\\ \cline{3-7}&& PrivateSQL & 744.56\%  & 0.00\% & 31590 & 0.163\\ \cline{3-7}\hline$\qstar$ &1369 & TSensDP & 1.77\%  & 0.00\% & 38 & 0.569\\ \cline{3-7}&& PrivateSQL & 599589.38\%  & 0.00\% & 8405100 & 0.628\\ \cline{3-7}\hline\end{ta

In [ ]:
def gen_dp_oracle_table_latex(queries, qtype, report_DP_limits, report_sens, measure='relerror'):
    import numpy as np
    algo = 'TSensDP'
    df = report_DP_limits.copy()
    df['time'] = df['avg_time']
    df.loc[df['nosy_ans'] < 0, 'nosy_ans'] = 0 
    df['relerror'] = np.absolute(df['nosy_ans'] - df['true_ans']) / df['true_ans']
    limits = sorted(df[df['query'].str.startswith("q_%s"%qtype)]['limit'].unique())
    tex_template = open('./result/table_dp_%s.tex.template'%'oracle', 'r').read()
    filler = ''
    filler += '\multirow{2}{*}{setting} & \multirow{2}{*}{LS} & \multicolumn{6}{c|}{$\ell$} \\\\ \\cline{3-8} \n&'
    for limit in limits:
        filler += '& %d\t'%limit
    filler += '\\\\ \\hline \n'
    for i in range(4):
        all_table_tstar = report_sens.query('algo=="TSens" & query=="{qname}"'.format(qname=queries[qtype][i]))['all_table_tstar'].to_list()[0]
        all_table_tstar = eval(re.sub(r'((tri_)?source.*?),', r"'\1',", all_table_tstar))
        for table_tstar in all_table_tstar:
            if table_tstar[0] == df.query('algo=="{algo}" & query=="{qname}"'.format(algo=algo, qname=queries[qtype][i]))['reln'].to_list()[0]:
                LS = table_tstar[2]
        filler += '$s_%d$ & %d\t'%(i+1, LS)
        for limit in limits:
            q = 'algo=="{algo}" & limit=="{limit}" & query=="{qname}"'.format(algo=algo, qname=queries[qtype][i], limit=limit)
            val = df.query(q)[measure].to_list()[0]
            filler += '& {val:.2f}\\%\t'.format(val=val*100)
        filler += '\\\\ \hline\n'
    table_tex = tex_template.replace('<filler>', filler).replace('<qtype>', qtype + ' queries')
    return table_tex

In [ ]:
for qtype in ['triangle', '4path', '4cycle', '4star']:
    print(gen_dp_oracle_table_latex(queries, qtype, report_DP_limits, report_sens))
    print()